# Funnel sur l’outil de calcul de l’indemnité conventionnelle

**Objectif** : En filtrant par session (ne pas avoir plusieurs fois un même event pour un même utilisateur) on voudrait avoir le nombre d’utilisateurs présents à chaque étape

Ceci doit nous permettre d’avoir le taux de complétion sur cette période

**Attention** : Les utilisateurs ne passent pas tous par toutes les étapes (certains ne passent pas par l’étape « infos » d’autres ne passent pas par l’étape « salaires »)
=> Ça ne change rien pour l’analyse, c’est pour l’interprétation après

**Les events à utiliser** : 
- event_category : outil
- event_action : view_step_Indemnité de licenciement
- event_name :
  - start
  - contrat_travail
  - info_cc
  - infos
  - anciennete
  - salaires
  - results

## On commence par le chargement des données

In [ ]:
import pandas as pd
import numpy as np
import time
import json
import math
from tqdm.notebook import tqdm
import plotly.express as px
import plotly.graph_objects as go
from datetime import datetime
from src.elasticsearch_connector import ElasticsearchConnector
#from typing import List, Dict

tqdm.pandas()


pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 100)
pd.set_option('display.width', 200)

## Si on veut lire en requêtant Elastic Search

In [ ]:
es_connector = ElasticsearchConnector(env='monolog')

QUERY = {
  "query": {
    "bool": { 
      "must": [
        {
          "prefix": {
            "url": "https://code.travail.gouv.fr/outils/indemnite-licenciement" 
          }
        },
        {
          "range": {
           "logfile": {
              "gte": "2023-01-02",
              "lt": "2023-03-19"
           }
          }
        }
      ]
    }
  }
}

In [ ]:
logs = es_connector.execute_query(QUERY, "logs-new")

# Début de l'exploration du funnel pour l'indemnite licenciement
## 1. Funnel sur l’outil de calcul de l’indemnité conventionnelle : du 18/02 au 18/03

### Préparation des données

In [ ]:
def format_url_and_get_month_and_datetime_columns(df: pd.DataFrame):
    df['url'] = df['url'].str.split('#').str[0].str.split('?').str[0]
    df['month'] = pd.DatetimeIndex(df['logfile']).month
    df['datetime'] = df['timestamp'].apply(datetime.fromtimestamp)
    return df

In [ ]:
logs = format_url_and_get_month_and_datetime_columns(logs.copy())

In [ ]:
logs = logs.drop(columns=['timestamp', 'lastActionTimestamp', 'referrerName', 'referrerTypeName', 
                          'query', 'timeSpent', 'feedbackType', 'visited', 'recoSelection', 'recoType',
                          'suggestionPrefix', 'suggestionSelection', 'resultSelection', 'lastActionDateTime', 
                          'serverTimePretty'])

In [ ]:
logs_feb_mar = logs[logs['logfile'] > '2023-02-17'].copy()

### Comptage du nombre de visiteurs uniques à chacune des étapes du parcours

In [ ]:
logs_indem_licen_view_step = logs_feb_mar[logs_feb_mar['outilAction']=='view_step'].copy()

In [ ]:
unique_visits_by_event_indem_licen = logs_indem_licen_view_step.groupby(
    by=['outilEvent']
)[['idVisit']].agg('nunique')

unique_visits_by_event_indem_licen = unique_visits_by_event_indem_licen.rename(
    columns={"idVisit": "Nombre de visiteurs uniques"})

In [ ]:
step_outil_indemnite_licenciement = {
    'start': 0, 'contrat_travail': 1, 'info_cc': 2, 'infos': 3, 'anciennete': 4, 'salaires': 5, 'results': 6
} 

unique_visits_by_event_indem_licen\
    .sort_values(by=['outilEvent'], key=lambda x: x.map(step_outil_indemnite_licenciement))

## 2. Séparer ceux qui passent/ne passent pas par « infos » du funnel sur l’outil de calcul de l’indemnité conventionnelle : du 18/02 au 18/03

In [ ]:
logs_indem_licen_view_step = logs_feb_mar[(logs_feb_mar['outilAction']=='view_step')].copy()

In [ ]:
# On récupère la liste de visiteurs étant passé par l'étape infos
users_having_selected_a_cc = logs_indem_licen_view_step[
    logs_indem_licen_view_step['outilEvent'].apply(lambda x: x == 'infos')
].idVisit.unique()

users_having_selected_a_cc = set(users_having_selected_a_cc)

In [ ]:
# on va voir si nos utilisateurs ont selectionné une CC

logs_indem_licen_view_step['Est passé par l étape infos'] = logs_indem_licen_view_step.idVisit.progress_apply(
    lambda visiteur: 'Oui' if visiteur in users_having_selected_a_cc else 'Non')

In [ ]:
unique_visits_by_event_indem_licen = logs_indem_licen_view_step.groupby(
    by=['Est passé par l étape infos', 'outilEvent']
)[['idVisit']].agg('nunique')

unique_visits_by_event_indem_licen = unique_visits_by_event_indem_licen.rename(
    columns={"idVisit": "Nombre de visiteurs uniques"})

In [ ]:
step_outil_indemnite_licenciement = {
    'start': 0, 'contrat_travail': 1, 'info_cc': 2, 'infos': 3, 'anciennete': 4, 'salaires': 5, 'results': 6
} 

dict_selection_cc = {
    'Oui': 0, 'Non': 1
} 

unique_visits_by_event_indem_licen\
    .sort_values(by=['outilEvent'], key=lambda x: x.map(step_outil_indemnite_licenciement))\
    .sort_values(by=['Est passé par l étape infos'], key=lambda x: x.map(dict_selection_cc))

## 3. Funnel sur l’outil de calcul de l’indemnité légale : du 02/01/23 au 30/01/23

### Préparation des données

In [ ]:
logs_jan = logs[(logs['logfile'] > '2023-01-01') & (logs['logfile'] < '2023-02-01')].copy()

### Comptage du nombre de visiteurs uniques à chacune des étapes du parcours

In [ ]:
logs_indem_licen_view_step = logs_jan[logs_jan['outilAction']=='view_step'].copy()

In [ ]:
unique_visits_by_event_indem_licen = logs_indem_licen_view_step.groupby(
    by=['outilEvent']
)[['idVisit']].agg('nunique')

unique_visits_by_event_indem_licen = unique_visits_by_event_indem_licen.rename(
    columns={"idVisit": "Nombre de visiteurs uniques"})

In [ ]:
step_outil_indemnite_licenciement = {
    'start': 0, 'contrat_travail': 1, 'anciennete': 4, 'salaires': 5, 'results': 6
} 

unique_visits_by_event_indem_licen\
    .sort_values(by=['outilEvent'], key=lambda x: x.map(step_outil_indemnite_licenciement))

## 4. Taux de sélection de CC sur l’outil de calul de l’indemnité de licenciement conventionnelle  : du 18/02 au 18/03

On cherche a avoir le nombre de cc_select_p1 + cc_select_p2 sur le nombre total de visiteurs de la page info_cc de l'outil indemnité de licenciement conventionnelle.

Commencons par le calcul du dénominateur soit le nombre de visits uniques sur la page info_cc. 

In [ ]:
nb_visits_info_cc = logs_feb_mar[
    (logs_feb_mar['outilAction']=='view_step') &
    (logs_feb_mar['outilEvent']=='info_cc')
].idVisit.nunique()
nb_visits_info_cc

Calculons maintenant le numérateur.

In [ ]:
nb_select_cc = logs_feb_mar[
    (logs_feb_mar['type']=='cc_select_p1') | 
    (logs_feb_mar['type']=='cc_select_p2') 
].idVisit.nunique()
nb_select_cc